In [1]:
cd ~/Downloads/OLHWDB1.1trn/

/home/wangmeng/Downloads/OLHWDB1.1trn


In [2]:
f_object = open('1001.mpf', 'rb')

In [3]:
import struct

In [4]:
size_of_header = struct.unpack('I', f_object.read(4))[0]

In [5]:
format_code = struct.unpack('8s', f_object.read(8))[0]

In [6]:
illustr = f_object.read(size_of_header - 62)

In [7]:
code_type = f_object.read(20)

In [8]:
code_length = struct.unpack('h', f_object.read(2))[0]

In [9]:
data_type = f_object.read(20)

In [10]:
sample_number = struct.unpack('I', f_object.read(4))[0]

In [11]:
dimensionality = struct.unpack('I', f_object.read(4))[0]

In [12]:
label = f_object.read(code_length).decode('GBK')

In [13]:
label.encode('utf-8')

b'\xe8\xa7\x92'

In [14]:
data = struct.unpack(str(dimensionality) + 'B', f_object.read(dimensionality))

In [ ]:
size_of_header = f_object.read(1)

In [ ]:
size_of_header = size_of_header[0]

In [ ]:
f_object.read(3)

In [ ]:
format_code = f_object.read(8)

In [ ]:
format_code

In [ ]:
illustr = f_object.read(size_of_header - 62)

In [ ]:
code_type = f_object.read(20)

In [ ]:
code_type

In [ ]:
code_length = f_object.read(1)

In [ ]:
code_length = code_length[0]

In [ ]:
f_object.read(1)

In [ ]:
data_type = f_object.read(20)

In [ ]:
data_type

In [ ]:
sample_number = f_object.read(4)

In [ ]:
sample_number 

In [ ]:
dimensionality = f_object.read(4)

In [ ]:
dimensionality